In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter

%matplotlib qt

from tensorflow.keras.datasets import cifar10

# import module with layers
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model_module")

from adaptive_layer import *

In [2]:
tf.random.set_seed(5)
np.random.seed(5)

In [3]:
def illustrate_train(name, y_label, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    epochs = range(1, len(static_f)+1)
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    plt.title(name, fontsize="xx-large")

    ax.set_ylabel(y_label, fontsize="large")
    ax.set_xlabel('Epochs', fontsize="large")

    ax.plot(epochs, static_f, label='ReLU Function')
    ax.plot(epochs, adaptive_f, label='Swish Function')
    ax.plot(epochs, adaptive_nn, label='Adaptive NN')
    ax.plot(epochs, adaptive_conv_nn, label='Adaptive Conv NN')

    ax.legend()
    
    plt.grid()
    plt.show()

In [4]:
def illustrate_test(name, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    
    x = ["ReLU Function", "Swish Function", "Adaptive NN", "Adaptive Conv NN"]
    y = [static_f, adaptive_f, adaptive_nn, adaptive_conv_nn]
    
    plt.title(name, fontsize="xx-large")
    ax.set_ylabel("Accuracy", fontsize="large")
    
    ax.bar(x, y)
    
    plt.grid()
    plt.show()

In [5]:
workbook = xlsxwriter.Workbook('cifar10_measurements.xlsx')

In [6]:
def save_data(name, static_f_train, adaptive_f_train, adaptive_nn_train, adaptive_conv_nn_train, 
              static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test):
    
    epochs = len(static_f_train[0][:])
    worksheet = workbook.add_worksheet(name)
    
    worksheet.write(1, 1, name + " Training Loss")
    worksheet.write(1, 7, name + " Training Accuracy")
    worksheet.write(1, 13, name + " Test Accuracy")
    
    worksheet.write(2, 1, "Epochs")
    worksheet.write(2, 2, "ReLU Function")
    worksheet.write(2, 3, "Swish Function")
    worksheet.write(2, 4, "Adaptive NN")
    worksheet.write(2, 5, "Adaptive Conv NN")
    
    worksheet.write(2, 7, "Epochs")
    worksheet.write(2, 8, "ReLU Function")
    worksheet.write(2, 9, "Swish Function")
    worksheet.write(2, 10, "Adaptive NN")
    worksheet.write(2, 11, "Adaptive Conv NN")
    
    worksheet.write(2, 13, "ReLU Function")
    worksheet.write(2, 14, "Swish Function")
    worksheet.write(2, 15, "Adaptive NN")
    worksheet.write(2, 16, "Adaptive Conv NN")
    
    worksheet.write(3, 13, static_f_test)
    worksheet.write(3, 14, adaptive_f_test)
    worksheet.write(3, 15, adaptive_nn_test)
    worksheet.write(3, 16, adaptive_conv_nn_test)
    
    for i in range(0, epochs):
        worksheet.write(3+i, 1, i+1)
        worksheet.write(3+i, 2, static_f_train[0][i])
        worksheet.write(3+i, 3, adaptive_f_train[0][i])
        worksheet.write(3+i, 4, adaptive_nn_train[0][i])
        worksheet.write(3+i, 5, adaptive_conv_nn_train[0][i])
        
        worksheet.write(3+i, 7, i+1)
        worksheet.write(3+i, 8, static_f_train[1][i])
        worksheet.write(3+i, 9, adaptive_f_train[1][i])
        worksheet.write(3+i, 10, adaptive_nn_train[1][i])
        worksheet.write(3+i, 11, adaptive_conv_nn_train[1][i])
    
    workbook.close()

In [7]:
# dataset

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train/255
x_test = x_test/255
x_train = np.reshape(x_train, (x_train.shape[0], -1))
x_test = np.reshape(x_test, (x_test.shape[0], -1))
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [8]:
EPOCHS = 30
name = "Cifar10"

# measurement repetitions
flag = measurements = 2

static_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_nn_hist = np.zeros(shape=[2, EPOCHS])
adaptive_conv_nn_hist = np.zeros(shape=[2, EPOCHS])

static_f_test = 0
adaptive_f_test = 0
adaptive_nn_test = 0
adaptive_conv_nn_test = 0

In [29]:
static_f_model = keras.Sequential([
    keras.layers.Dense(400, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])

In [30]:
adaptive_f_model = keras.Sequential([
    keras.layers.Dense(400, activation=tf.nn.swish),
    keras.layers.Dense(50, activation=tf.nn.swish),
    keras.layers.Dense(10, activation="softmax"),
])

In [31]:
# Adaptive Layer
adaptive_nn_model = AdaptiveModel([
    AdaptiveLayer(400, structure=[2, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.leaky_relu, skip_w=1.5), 
    AdaptiveLayer(50, structure=[4, 4, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.leaky_relu, skip_w=0.9), 
    AdaptiveLayer(10, structure=[8, 4, 1], inner_hidden_activation=tf.nn.relu, 
                 inner_out_activation=tf.nn.softmax, skip_w=0.9)
])

In [32]:
# Adaptive Layer based on 1D Conv
adaptive_conv_nn_model = AdaptiveModel([
    AdaptiveLayerConv(400, structure=[32], split=1, noise=0, skip_w=1.5, 
                      inner_hidden_activation=tf.nn.leaky_relu, inner_out_activation=tf.nn.leaky_relu), 
    AdaptiveLayerConv(50, structure=[64], split=2, noise=0, skip_w=0.9, 
                      inner_hidden_activation=tf.nn.leaky_relu, inner_out_activation=tf.nn.leaky_relu), 
    AdaptiveLayerConv(10, structure=[128], split=3, noise=0, skip_w=0.9, 
                      inner_hidden_activation=tf.nn.leaky_relu, inner_out_activation=tf.nn.softmax) 
])

In [33]:
static_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [34]:
adaptive_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [35]:
adaptive_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [36]:
adaptive_conv_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [37]:
hist = static_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/30
782/782 [==============================] - 3s 3ms/step - loss: 2.0802 - accuracy: 0.2039
Epoch 2/30
782/782 [==============================] - 3s 3ms/step - loss: 1.8564 - accuracy: 0.3165
Epoch 3/30
782/782 [==============================] - 3s 3ms/step - loss: 1.7343 - accuracy: 0.3726
Epoch 4/30
782/782 [==============================] - 3s 3ms/step - loss: 1.6744 - accuracy: 0.3984
Epoch 5/30
782/782 [==============================] - 3s 3ms/step - loss: 1.6277 - accuracy: 0.4142
Epoch 6/30
782/782 [==============================] - 3s 3ms/step - loss: 1.5840 - accuracy: 0.4321
Epoch 7/30
782/782 [==============================] - 3s 3ms/step - loss: 1.5460 - accuracy: 0.4458
Epoch 8/30
782/782 [==============================] - 3s 3ms/step - loss: 1.5187 - accuracy: 0.4568
Epoch 9/30
782/782 [==============================] - 3s 3ms/step - loss: 1.4879 - accuracy: 0.4665
Epoch 10/30
782/782 [==============================] - 3s 3ms/step - loss: 1.4636 - accuracy: 0.4771

In [38]:
static_f_hist[0][:] += np.array(hist.history["loss"])/measurements
static_f_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [39]:
hist = adaptive_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/30
782/782 [==============================] - 3s 4ms/step - loss: 1.9795 - accuracy: 0.2823
Epoch 2/30
782/782 [==============================] - 3s 4ms/step - loss: 1.7383 - accuracy: 0.3764
Epoch 3/30
782/782 [==============================] - 3s 4ms/step - loss: 1.6381 - accuracy: 0.4135
Epoch 4/30
782/782 [==============================] - 3s 4ms/step - loss: 1.5835 - accuracy: 0.4326
Epoch 5/30
782/782 [==============================] - 3s 4ms/step - loss: 1.5314 - accuracy: 0.4501
Epoch 6/30
782/782 [==============================] - 3s 4ms/step - loss: 1.4970 - accuracy: 0.4632
Epoch 7/30
782/782 [==============================] - 3s 4ms/step - loss: 1.4642 - accuracy: 0.4726
Epoch 8/30
782/782 [==============================] - 3s 4ms/step - loss: 1.4403 - accuracy: 0.4845
Epoch 9/30
782/782 [==============================] - 3s 4ms/step - loss: 1.4172 - accuracy: 0.4928
Epoch 10/30
782/782 [==============================] - 3s 4ms/step - loss: 1.3882 - accuracy: 0.5010

In [40]:
adaptive_f_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_f_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [41]:
hist = adaptive_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/30
782/782 [==============================] - 4s 5ms/step - loss: 2.2109 - accuracy: 0.2304
Epoch 2/30
782/782 [==============================] - 4s 5ms/step - loss: 1.8494 - accuracy: 0.3424
Epoch 3/30
782/782 [==============================] - 4s 5ms/step - loss: 1.7244 - accuracy: 0.3884
Epoch 4/30
782/782 [==============================] - 4s 5ms/step - loss: 1.6518 - accuracy: 0.4122
Epoch 5/30
782/782 [==============================] - 4s 5ms/step - loss: 1.6016 - accuracy: 0.4291
Epoch 6/30
782/782 [==============================] - 4s 5ms/step - loss: 1.5526 - accuracy: 0.4463
Epoch 7/30
782/782 [==============================] - 4s 5ms/step - loss: 1.5129 - accuracy: 0.4577
Epoch 8/30
782/782 [==============================] - 4s 5ms/step - loss: 1.4822 - accuracy: 0.4689
Epoch 9/30
782/782 [==============================] - 4s 5ms/step - loss: 1.4485 - accuracy: 0.4800
Epoch 10/30
782/782 [==============================] - 4s 5ms/step - loss: 1.4268 - accuracy: 0.4914

In [42]:
adaptive_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_nn_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [43]:
hist = adaptive_conv_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/30
782/782 [==============================] - 5s 6ms/step - loss: 1.8614 - accuracy: 0.3212
Epoch 2/30
782/782 [==============================] - 5s 6ms/step - loss: 1.6744 - accuracy: 0.3967
Epoch 3/30
782/782 [==============================] - 5s 7ms/step - loss: 1.5884 - accuracy: 0.4289
Epoch 4/30
782/782 [==============================] - 5s 7ms/step - loss: 1.5368 - accuracy: 0.4526
Epoch 5/30
782/782 [==============================] - 5s 7ms/step - loss: 1.4944 - accuracy: 0.4630
Epoch 6/30
782/782 [==============================] - 5s 6ms/step - loss: 1.4582 - accuracy: 0.4771
Epoch 7/30
782/782 [==============================] - 5s 7ms/step - loss: 1.4277 - accuracy: 0.4884
Epoch 8/30
782/782 [==============================] - 5s 7ms/step - loss: 1.3980 - accuracy: 0.4986
Epoch 9/30
782/782 [==============================] - 5s 7ms/step - loss: 1.3653 - accuracy: 0.5107
Epoch 10/30
782/782 [==============================] - 5s 7ms/step - loss: 1.3410 - accuracy: 0.5218

In [44]:
adaptive_conv_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_conv_nn_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [45]:
static_f_test += static_f_model.evaluate(x_test, y_test)[1]/measurements

313/313 [==============================] - 0s 985us/step - loss: 1.4666 - accuracy: 0.4878


In [46]:
adaptive_f_test += adaptive_f_model.evaluate(x_test, y_test)[1]/measurements

313/313 [==============================] - 0s 923us/step - loss: 1.4823 - accuracy: 0.4919


In [47]:
adaptive_nn_test += adaptive_nn_model.evaluate(x_test, y_test)[1]/measurements

313/313 [==============================] - 0s 1ms/step - loss: 1.4375 - accuracy: 0.5141


In [48]:
adaptive_conv_nn_test += adaptive_conv_nn_model.evaluate(x_test, y_test)[1]/measurements
flag -= 1
if(flag == 0):
    print("---------FINISH---------")

313/313 [==============================] - 1s 1ms/step - loss: 1.4192 - accuracy: 0.5185
---------FINISH---------


In [49]:
print("Static Function model\nTest accuracy : ", static_f_test)
print("\nAdaptive Function model\nTest accuracy : ", adaptive_f_test)
print("\nAdaptive NN model\nTest accuracy : ", adaptive_nn_test)
print("\nAdaptive Conv NN model\nTest accuracy : ", adaptive_conv_nn_test)

Static Function model
Test accuracy :  0.49050000309944153

Adaptive Function model
Test accuracy :  0.4943999946117401

Adaptive NN model
Test accuracy :  0.5115500092506409

Adaptive Conv NN model
Test accuracy :  0.5206499993801117


In [50]:
illustrate_train(name+" Training Loss", "Loss", static_f_hist[0][:], 
                 adaptive_f_hist[0][:], adaptive_nn_hist[0][:], adaptive_conv_nn_hist[0][:])

In [51]:
illustrate_train(name + " Training Accuracy", "Accuracy", static_f_hist[1][:], 
                 adaptive_f_hist[1][:], adaptive_nn_hist[1][:], adaptive_conv_nn_hist[1][:])

In [52]:
illustrate_test(name + " Test Accuracy", 
                static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)

In [1]:
save_data(name, static_f_hist, adaptive_f_hist, adaptive_nn_hist, 
          adaptive_conv_nn_hist, static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)

NameError: name 'save_data' is not defined